# Save Lang2motion transformer model

In [ ]:
// for local development
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt")' Datasets TranslationModels TextModels ModelSupport SummaryWriter LangMotionModels Checkpoints

In [ ]:
import TensorFlow
import TextModels
import TranslationModels
import Foundation
import ModelSupport
import Datasets
import SummaryWriter
import LangMotionModels
import Checkpoints

## Set training params

In [ ]:
let maxTextSequenceLength =  20
let maxMotionLength =  100

In [ ]:
let dataURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/")

## Instantiate model

In [ ]:
/// instantiate text processor
let vocabularyURL = dataURL.appendingPathComponent("vocab.txt")
let vocabulary: Vocabulary = try! Vocabulary(fromFile: vocabularyURL)
let tokenizer: Tokenizer = BERTTokenizer(vocabulary: vocabulary, caseSensitive: false, unknownToken: "[UNK]", maxTokenLength: nil)
let textProcessor = TextProcessor2(vocabulary: vocabulary, tokenizer: tokenizer, maxTextSequenceLength: maxTextSequenceLength, maxMotionLength: maxMotionLength)

/// instantiate model
let vocabSize = vocabulary.count
let nbJoints = 47 // TODO: get value from dataset
let nbMixtures = 20
let layerCount: Int = 6
let modelSize: Int = 256
let feedForwardSize: Int = 1024
let headCount: Int = 8
let dropoutProbability: Double = 0.1

var model = LangMotionTransformer(
    vocabSize: vocabSize, 
    nbJoints: nbJoints,
    nbMixtures: nbMixtures,
    layerCount: layerCount, 
    modelSize: modelSize, 
    feedForwardSize: feedForwardSize, 
    headCount: headCount, 
    dropoutProbability: dropoutProbability
)

## play with writer and reader

In [ ]:
let temporaryDirectory = dataURL.appendingPathComponent("CheckpointsTests", isDirectory: true)

## save parts of the model

In [ ]:
public protocol ExportableLayer {
    var nameMappings: [String: String] { get }
}

In [ ]:
extension LangMotionTransformer: ExportableLayer {
    public var nameMappings: [String: String] {        
        [
            "encoder": "encoder",
            "decoder": "decoder",
            "positionalEncoding": "positionalEncoding",
            "motionDense": "motionDense",
            "sourceEmbed": "sourceEmbed",
            "mixtureModel": "mixtureModel",
        ]
        // modelSize: Int
        // nbJoints: Int
        // nbMixtures: Int
    }
}

In [ ]:
extension Encoder: ExportableLayer {
    public var nameMappings: [String: String] { 
        [
            "layers": "layers",
            "norm": "norm"
        ]
    }
}

In [ ]:
extension Decoder: ExportableLayer {
    public var nameMappings: [String: String] { 
        [
            "layers": "layers",
            "norm": "norm"
        ]
    }
}

In [ ]:
extension Array: ExportableLayer {
    public var nameMappings: [String: String] { ["h": "\(type(of:self))".components(separatedBy: ["<", ">"])[1] + "_h" ] }
}

In [ ]:
extension MotionGaussianMixtureModel: ExportableLayer {
    public var nameMappings: [String: String] {
        [
            "linearMixtureMeans": "linearMixtureMeans",
            "linearMixtureVars": "linearMixtureVars",
            "linearMixtureWeights": "linearMixtureWeights",
            "linearStop": "linearStop",
            // inputSize: Int
            // nbJoints: Int
            // nbMixtures: Int
            // outputSize: Int
        ] 
    }
}

In [ ]:
extension Embedding: ExportableLayer {
    public var nameMappings: [String: String] { ["weight": "weight"] }
}

In [ ]:
extension LayerNorm: ExportableLayer {
    public var nameMappings: [String: String] { ["offset": "offset", "scale": "scale"] }
}

In [ ]:
extension Dense: ExportableLayer {
    public var nameMappings: [String: String] { ["weight": "weight", "bias": "bias"] }
}

In [ ]:
extension MultiHeadAttention: ExportableLayer {
    public var nameMappings: [String: String] { 
        [
            // sourceSize: Int
            // targetSize: Int
            // headCount: Int
            // eadSize: Int
            // queryActivation: Activation<Scalar>
            // keyActivation: Activation<Scalar>
            // valueActivation: Activation<Scalar>
            // matrixResult: Bool

            "queryWeight": "queryWeight",
            "queryBias": "queryBias",
            "keyWeight": "keyWeight",
            "keyBias": "keyBias",
            "valueWeight": "valueWeight",
            "valueBias": "valueBias",
            // attentionDropout: Dropout<Scalar>
        ] 
    }
}

In [ ]:
extension TransformerEncoderLayer2: ExportableLayer {
    public var nameMappings: [String: String] { 
        [
            "selfAttention": "selfAttention",
            "feedForward": "feedForward",
            "sublayers": "sublayers",
        ] 
    }
}

In [ ]:
extension PositionwiseFeedForward: ExportableLayer {
    public var nameMappings: [String: String] { 
        [
            "dense1": "dense1",
            "dense2": "dense2",
            "dropout": "dropout",
        ] 
    }
}

In [ ]:
extension SublayerConnection: ExportableLayer {
    public var nameMappings: [String: String] { 
        [
            "norm": "norm",
            "dropout": "dropout",
        ] 
    }
}

In [ ]:
extension TransformerDecoderLayer: ExportableLayer {
    public var nameMappings: [String: String] { 
        [
            "selfAttention": "selfAttention",
            "sourceAttention": "sourceAttention",
            "feedForward": "feedForward",
            "sublayers": "sublayers",            
        ] 
    }
}

In [ ]:
public func recursivelyObtainTensors(
    _ obj: Any, scope: String? = nil, tensors: inout [String: Tensor<Float>], separator: String
) {
    
    let m = Mirror(reflecting: obj)
    let nameMappings: [String: String]
    if let exportableLayer = obj as? ExportableLayer {
        nameMappings = exportableLayer.nameMappings
    } else {
        if (obj is Int) || (obj is Bool) || (obj is Tensor<Float>) || 
           (obj is Double) || (obj is Float) || (obj is Dropout<Float>) ||
           (obj is Parameter<Float>) || (obj is PositionalEncoding)
        {}
        else {
            let s = "\(scope!) -> \(type(of:obj))"
            if !s.contains("Tensor") {
                print(s)
            }
        }
        nameMappings = [:]
    }

    var repeatedLabels: [String: Int] = [:]
    func suffix(for label: String) -> String {
        if let currentSuffix = repeatedLabels[label] {
            repeatedLabels[label] = currentSuffix + 1
            return "\(currentSuffix + 1)"
        } else {
            repeatedLabels[label] = 0
            return "0"
        }
    }

    let hasSuffix = (m.children.first?.label == nil)

    var path = scope
    for child in m.children {
        let label = child.label ?? "h"

        if let remappedLabel = nameMappings[label] {
            let labelSuffix = hasSuffix ? suffix(for: remappedLabel) : ""
            let conditionalSeparator = remappedLabel == "" ? "" : separator

            path = (scope != nil ? scope! + conditionalSeparator : "") + remappedLabel + labelSuffix
            if let tensor = child.value as? Tensor<Float> {
                tensors[path!] = tensor
            }
        }
        recursivelyObtainTensors(child.value, scope: path, tensors: &tensors, separator: separator)
    }
}

In [ ]:
func writeCheckpoint(to location: URL, name: String) throws {
    var tensors = [String: Tensor<Float>]()
    recursivelyObtainTensors(model, scope: "model", tensors: &tensors, separator: "/")
    
    tensors.keys.sorted().map {print($0)}
    
    let writer = CheckpointWriter(tensors: tensors)
    try writer.write(to: location, name: name)
}

In [ ]:
writeCheckpoint(to: temporaryDirectory, name: "model1")